In [66]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
import seaborn as sns

import random

random.seed(10)



In [67]:
i=10

In [68]:
output_dir = r"/Users/juju/Library/Mobile Documents/com~apple~CloudDocs/ta_project/data/rdd_data"
df = pd.read_csv(os.path.join(output_dir, r"rdd_data_{}.csv".format(i)))

In [69]:
df["operation_year"] = df["Operation"]+df["year"].astype(str)

In [70]:
df.groupby("Operation").count()

,year_month,shareid,Year Month,Occ,ADR,RevPAR,Market,Class,Location,Open Date,...,ln_revpar_next_1m,year,month,displayed_rating,round_up,market_year_month,time_period,rounding_threshold,distance_to_cutoff,operation_year
Operation,,,,,,,,,,,,,,,,,,,,,
Chain,26905,26905,26905,26905,26905,26905,26905,26905,26905,26905,...,26900,26905,26905,26905,26905,26905,26905,26905,26905,26905
Independent,3800,3800,3800,3800,3800,3800,3800,3800,3800,3792,...,3790,3800,3800,3800,3800,3800,3800,3800,3800,3800


In [42]:
# get displayed rating
df['displayed_rating'] = np.select([
    (df["accum_rating"] < 1.25),
    (df["accum_rating"] >= 1.25) & (df["accum_rating"] < 1.75),
    (df["accum_rating"] >= 1.75) & (df["accum_rating"] < 2.25),
    (df["accum_rating"] >= 2.25) & (df["accum_rating"] < 2.75),
    (df["accum_rating"] >= 2.75) & (df["accum_rating"] < 3.25),
    (df["accum_rating"] >= 3.25) & (df["accum_rating"] < 3.75),
    (df["accum_rating"] >= 3.75) & (df["accum_rating"] < 4.25),
    (df["accum_rating"] >= 4.25) & (df["accum_rating"] < 4.75),
    (df["accum_rating"] >= 4.75) & (df["accum_rating"] < 5)
], [1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5])
df["round_up"] = df["accum_rating"]<=df["displayed_rating"]

In [43]:
df["accum_rating_sq"] = df["accum_rating"]**2

In [44]:
# Dependent variable
dep_var_name_1 = "ln_occ"
dep_var_name_2 = "ln_adr"
dep_var_name_3 = "ln_revpar"

In [45]:
# Create market year-month dummy

df["market_year_month"] = df["year_month"].astype(str) + df["Market"]

In [46]:
df = df[~df[dep_var_name_1].isna()].reset_index()

In [47]:
y_1 = df[dep_var_name_1]
y_2 = df[dep_var_name_2]
y_3 = df[dep_var_name_3]

In [48]:
df["time_period"].unique()

array(['2000-2004', '2005-2009', '2010-2014', '2015-2019'], dtype=object)

In [49]:

OH = OneHotEncoder(handle_unknown='ignore')

fe_var = ["time_period"]

x_fe = OH.fit_transform(df[fe_var]).toarray()
fe_var_cols = OH.fit(df[fe_var]).get_feature_names_out().tolist()

fe_var_df = pd.DataFrame(x_fe, columns=fe_var_cols)

con_var_list = ["accum_rating","round_up","accum_rating_sq"]
#scaler = preprocessing.StandardScaler().fit(df_con_var[con_var_list_1])
#x = scaler.transform(df_con_var[con_var_list_1])
con_var_df = df[con_var_list]


x_temp = pd.concat([con_var_df,fe_var_df], axis=1).astype(float)

x_temp["2000-2004_round_up"] = x_temp["time_period_2000-2004"]*x_temp["round_up"]
x_temp["2005-2009_round_up"] = x_temp["time_period_2005-2009"]*x_temp["round_up"]
x_temp["2010-2014_round_up"] = x_temp["time_period_2010-2014"]*x_temp["round_up"]
x_temp["2015-2019_round_up"] = x_temp["time_period_2015-2019"]*x_temp["round_up"]

x_temp["2000-2004_accum_rating"] = x_temp["time_period_2000-2004"]*x_temp["accum_rating"]
x_temp["2005-2009_accum_rating"] = x_temp["time_period_2005-2009"]*x_temp["accum_rating"]
x_temp["2010-2014_accum_rating"] = x_temp["time_period_2010-2014"]*x_temp["accum_rating"]
x_temp["2015-2019_accum_rating"] = x_temp["time_period_2015-2019"]*x_temp["accum_rating"]

x_temp["2000-2004_accum_rating_sq"] = x_temp["time_period_2000-2004"]*x_temp["accum_rating_sq"]
x_temp["2005-2009_accum_rating_sq"] = x_temp["time_period_2005-2009"]*x_temp["accum_rating_sq"]
x_temp["2010-2014_accum_rating_sq"] = x_temp["time_period_2010-2014"]*x_temp["accum_rating_sq"]
x_temp["2015-2019_accum_rating_sq"] = x_temp["time_period_2015-2019"]*x_temp["accum_rating_sq"]
'''
#x_temp["2001_round_up"] = x_temp["year_2001"]*x_temp["round_up"]
#x_temp["2002_round_up"] = x_temp["year_2002"]*x_temp["round_up"]
x_temp["2003_round_up"] = x_temp["year_2003"]*x_temp["round_up"]
x_temp["2004_round_up"] = x_temp["year_2004"]*x_temp["round_up"]
x_temp["2005_round_up"] = x_temp["year_2005"]*x_temp["round_up"]
x_temp["2006_round_up"] = x_temp["year_2006"]*x_temp["round_up"]
x_temp["2007_round_up"] = x_temp["year_2007"]*x_temp["round_up"]
x_temp["2008_round_up"] = x_temp["year_2008"]*x_temp["round_up"]
x_temp["2009_round_up"] = x_temp["year_2009"]*x_temp["round_up"]
x_temp["2010_round_up"] = x_temp["year_2010"]*x_temp["round_up"]
x_temp["2011_round_up"] = x_temp["year_2011"]*x_temp["round_up"]
x_temp["2012_round_up"] = x_temp["year_2012"]*x_temp["round_up"]
x_temp["2013_round_up"] = x_temp["year_2013"]*x_temp["round_up"]
x_temp["2014_round_up"] = x_temp["year_2014"]*x_temp["round_up"]
x_temp["2015_round_up"] = x_temp["year_2015"]*x_temp["round_up"]
x_temp["2016_round_up"] = x_temp["year_2016"]*x_temp["round_up"]
x_temp["2017_round_up"] = x_temp["year_2017"]*x_temp["round_up"]
x_temp["2018_round_up"] = x_temp["year_2018"]*x_temp["round_up"]
x_temp["2019_round_up"] = x_temp["year_2019"]*x_temp["round_up"]


x_temp["2003_accum_rating"] = x_temp["year_2003"]*x_temp["accum_rating"]
x_temp["2004_accum_rating"] = x_temp["year_2004"]*x_temp["accum_rating"]
x_temp["2005_accum_rating"] = x_temp["year_2005"]*x_temp["accum_rating"]
x_temp["2006_accum_rating"] = x_temp["year_2006"]*x_temp["accum_rating"]
x_temp["2007_accum_rating"] = x_temp["year_2007"]*x_temp["accum_rating"]
x_temp["2008_accum_rating"] = x_temp["year_2008"]*x_temp["accum_rating"]
x_temp["2009_accum_rating"] = x_temp["year_2009"]*x_temp["accum_rating"]
x_temp["2010_accum_rating"] = x_temp["year_2010"]*x_temp["accum_rating"]
x_temp["2011_accum_rating"] = x_temp["year_2011"]*x_temp["accum_rating"]
x_temp["2012_accum_rating"] = x_temp["year_2012"]*x_temp["accum_rating"]
x_temp["2013_accum_rating"] = x_temp["year_2013"]*x_temp["accum_rating"]
x_temp["2014_accum_rating"] = x_temp["year_2014"]*x_temp["accum_rating"]
x_temp["2015_accum_rating"] = x_temp["year_2015"]*x_temp["accum_rating"]
x_temp["2016_accum_rating"] = x_temp["year_2016"]*x_temp["accum_rating"]
x_temp["2017_accum_rating"] = x_temp["year_2017"]*x_temp["accum_rating"]
x_temp["2018_accum_rating"] = x_temp["year_2018"]*x_temp["accum_rating"]
x_temp["2019_accum_rating"] = x_temp["year_2019"]*x_temp["accum_rating"]


x_temp["2003_accum_rating_sq"] = x_temp["year_2003"]*x_temp["accum_rating_sq"]
x_temp["2004_accum_rating_sq"] = x_temp["year_2004"]*x_temp["accum_rating_sq"]
x_temp["2005_accum_rating_sq"] = x_temp["year_2005"]*x_temp["accum_rating_sq"]
x_temp["2006_accum_rating_sq"] = x_temp["year_2006"]*x_temp["accum_rating_sq"]
x_temp["2007_accum_rating_sq"] = x_temp["year_2007"]*x_temp["accum_rating_sq"]
x_temp["2008_accum_rating_sq"] = x_temp["year_2008"]*x_temp["accum_rating_sq"]
x_temp["2009_accum_rating_sq"] = x_temp["year_2009"]*x_temp["accum_rating_sq"]
x_temp["2010_accum_rating_sq"] = x_temp["year_2010"]*x_temp["accum_rating_sq"]
x_temp["2011_accum_rating_sq"] = x_temp["year_2011"]*x_temp["accum_rating_sq"]
x_temp["2012_accum_rating_sq"] = x_temp["year_2012"]*x_temp["accum_rating_sq"]
x_temp["2013_accum_rating_sq"] = x_temp["year_2013"]*x_temp["accum_rating_sq"]
x_temp["2014_accum_rating_sq"] = x_temp["year_2014"]*x_temp["accum_rating_sq"]
x_temp["2015_accum_rating_sq"] = x_temp["year_2015"]*x_temp["accum_rating_sq"]
x_temp["2016_accum_rating_sq"] = x_temp["year_2016"]*x_temp["accum_rating_sq"]
x_temp["2017_accum_rating_sq"] = x_temp["year_2017"]*x_temp["accum_rating_sq"]
x_temp["2018_accum_rating_sq"] = x_temp["year_2018"]*x_temp["accum_rating_sq"]
x_temp["2019_accum_rating_sq"] = x_temp["year_2019"]*x_temp["accum_rating_sq"]
'''

'\n#x_temp["2001_round_up"] = x_temp["year_2001"]*x_temp["round_up"]\n#x_temp["2002_round_up"] = x_temp["year_2002"]*x_temp["round_up"]\nx_temp["2003_round_up"] = x_temp["year_2003"]*x_temp["round_up"]\nx_temp["2004_round_up"] = x_temp["year_2004"]*x_temp["round_up"]\nx_temp["2005_round_up"] = x_temp["year_2005"]*x_temp["round_up"]\nx_temp["2006_round_up"] = x_temp["year_2006"]*x_temp["round_up"]\nx_temp["2007_round_up"] = x_temp["year_2007"]*x_temp["round_up"]\nx_temp["2008_round_up"] = x_temp["year_2008"]*x_temp["round_up"]\nx_temp["2009_round_up"] = x_temp["year_2009"]*x_temp["round_up"]\nx_temp["2010_round_up"] = x_temp["year_2010"]*x_temp["round_up"]\nx_temp["2011_round_up"] = x_temp["year_2011"]*x_temp["round_up"]\nx_temp["2012_round_up"] = x_temp["year_2012"]*x_temp["round_up"]\nx_temp["2013_round_up"] = x_temp["year_2013"]*x_temp["round_up"]\nx_temp["2014_round_up"] = x_temp["year_2014"]*x_temp["round_up"]\nx_temp["2015_round_up"] = x_temp["year_2015"]*x_temp["round_up"]\nx_tem

In [50]:

OH = OneHotEncoder(handle_unknown='ignore')

fe_var = ["market_year_month","shareid"]

x_fe = OH.fit_transform(df[fe_var]).toarray()
fe_var_cols = OH.fit(df[fe_var]).get_feature_names_out().tolist()

fe_var_df_1 = pd.DataFrame(x_fe, columns=fe_var_cols)
#.drop(["shareid_75909","year_month_2017-11-01"],axis=1)

con_var_list = ["accum_rating"]
#scaler = preprocessing.StandardScaler().fit(df_con_var[con_var_list_1])
#x = scaler.transform(df_con_var[con_var_list_1])
con_var_df_1 = df[con_var_list]

x_1 = pd.concat([df[["round_up"]],con_var_df_1,fe_var_df_1], axis=1).astype(float)
x_1 = sm.add_constant(x_1)

In [51]:
con_var_list = ["accum_rating","accum_rating_sq"]
con_var_df_2 = df[con_var_list]
x_2= pd.concat([df[["round_up"]],con_var_df_2,fe_var_df_1], axis=1).astype(float)
x_2 = sm.add_constant(x_2)

In [156]:
from statsmodels.iolib.summary2 import summary_col
from statsmodels.sandbox.regression.gmm import IV2SLS

model_1 = sm.OLS(y_1, x_1)
results_1 = model_1.fit()

model_2 = sm.OLS(y_1, x_2)
results_2 = model_2.fit()

model_3 = sm.OLS(y_2, x_1)
results_3 = model_3.fit()

model_4 = sm.OLS(y_2, x_2)
results_4 = model_4.fit()

model_5 = sm.OLS(y_3, x_1)
results_5 = model_5.fit()

model_6 = sm.OLS(y_3, x_2)
results_6 = model_6.fit()

In [157]:
res_main = summary_col([results_1, results_2,\
                       results_3, results_4,\
                       results_5, results_6], float_format = '%.3f', \
                  model_names = ["ln_Occ_jt (1)", 
                                 "ln_Occ_jt (2)",
                                 "ln_ADR_jt (1)",
                                 "ln_ADR_jt (2)",
                                 "ln_RevPAR_jt (1)",
                                 "ln_RevPAR_jt (2)",
                                 
                                 ],\
                 regressor_order = ["round_up",\
                                   "accum_rating",\
                                    "accum_rating_sq"
                                  ],\
                  info_dict={'N':lambda x: "{0:d}".format(int(x.nobs)),
                           'Adjusted R-squared':lambda x: "{:.2f}".format(x.rsquared_adj)},\
                  stars = True,drop_omitted=True)


beginningtex = """\\documentclass{report}
\\usepackage{booktabs}
\\begin{document}"""
endtex = "\end{document}"

output_dir = r"/Users/juju/Library/Mobile Documents/com~apple~CloudDocs/ta_project/output/ch1"
f = open(os.path.join(output_dir,'ch1_rdd_{}.tex'.format(i)), 'w')
f.write(beginningtex)
f.write(res_main.as_latex())
f.write(endtex)
f.close()

# By Year

In [52]:

    
con_var_df_2 = x_temp[["2000-2004_round_up", "2005-2009_round_up","2010-2014_round_up","2015-2019_round_up",\
        "2000-2004_accum_rating", "2005-2009_accum_rating","2010-2014_accum_rating","2015-2019_accum_rating",\
        "2000-2004_accum_rating_sq", "2005-2009_accum_rating_sq","2010-2014_accum_rating_sq","2015-2019_accum_rating_sq",\
       ]]

con_var_df_1 = x_temp[["2000-2004_round_up", "2005-2009_round_up","2010-2014_round_up","2015-2019_round_up",\
        "2000-2004_accum_rating", "2005-2009_accum_rating","2010-2014_accum_rating","2015-2019_accum_rating",\
       ]]



'''

con_var_df_1 = x_temp[["2003_accum_rating",\
                    "2004_accum_rating",\
                    "2005_accum_rating",\
                    "2006_accum_rating",\
                    "2007_accum_rating",\
                    "2008_accum_rating",\
                    "2009_accum_rating",\
                    "2010_accum_rating",\
                    "2011_accum_rating",\
                    "2012_accum_rating",\
                     "2013_accum_rating",\
                     "2014_accum_rating",\
                     "2015_accum_rating",\
                     "2016_accum_rating",\
                     "2017_accum_rating",\
                     "2018_accum_rating",\
                     "2019_accum_rating",\
         "2003_round_up",\
                    "2004_round_up",\
                    "2005_round_up",\
                    "2006_round_up",\
                    "2007_round_up",\
                    "2008_round_up",\
                    "2009_round_up",\
                    "2010_round_up",\
                    "2011_round_up",\
                    "2012_round_up",\
                     "2013_round_up",\
                     "2014_round_up",\
                     "2015_round_up",\
                     "2016_round_up",\
                     "2017_round_up",\
                     "2018_round_up",\
                     "2019_round_up",\
         
                    ]]
    '''

x_1 = pd.concat([con_var_df_1,fe_var_df_1], axis=1).astype(float)

In [53]:
'''
con_var_df_2 = x_temp[["2003_accum_rating",\
                    "2004_accum_rating",\
                    "2005_accum_rating",\
                    "2006_accum_rating",\
                    "2007_accum_rating",\
                    "2008_accum_rating",\
                    "2009_accum_rating",\
                    "2010_accum_rating",\
                    "2011_accum_rating",\
                    "2012_accum_rating",\
                     "2013_accum_rating",\
                     "2014_accum_rating",\
                     "2015_accum_rating",\
                     "2016_accum_rating",\
                     "2017_accum_rating",\
                     "2018_accum_rating",\
                     "2019_accum_rating",\
         "2003_round_up",\
                    "2004_round_up",\
                    "2005_round_up",\
                    "2006_round_up",\
                    "2007_round_up",\
                    "2008_round_up",\
                    "2009_round_up",\
                    "2010_round_up",\
                    "2011_round_up",\
                    "2012_round_up",\
                     "2013_round_up",\
                     "2014_round_up",\
                     "2015_round_up",\
                     "2016_round_up",\
                     "2017_round_up",\
                     "2018_round_up",\
                     "2019_round_up",\
                       
                       "2003_accum_rating_sq",\
                    "2004_accum_rating_sq",\
                    "2005_accum_rating_sq",\
                    "2006_accum_rating_sq",\
                    "2007_accum_rating_sq",\
                    "2008_accum_rating_sq",\
                    "2009_accum_rating_sq",\
                    "2010_accum_rating_sq",\
                    "2011_accum_rating_sq",\
                    "2012_accum_rating_sq",\
                     "2013_accum_rating_sq",\
                     "2014_accum_rating_sq",\
                     "2015_accum_rating_sq",\
                     "2016_accum_rating_sq",\
                     "2017_accum_rating_sq",\
                     "2018_accum_rating_sq",\
                     "2019_accum_rating_sq"
         
                    ]]
 '''   

x_2 = pd.concat([con_var_df_2,fe_var_df_1], axis=1).astype(float)

In [54]:
from statsmodels.iolib.summary2 import summary_col
from statsmodels.sandbox.regression.gmm import IV2SLS

model_1 = sm.OLS(y_1, x_1)
results_1 = model_1.fit()

model_2 = sm.OLS(y_1, x_2)
results_2 = model_2.fit()

model_3 = sm.OLS(y_2, x_1)
results_3 = model_3.fit()

model_4 = sm.OLS(y_2, x_2)
results_4 = model_4.fit()

model_5 = sm.OLS(y_3, x_1)
results_5 = model_5.fit()

model_6 = sm.OLS(y_3, x_2)
results_6 = model_6.fit()

In [55]:
res_main = summary_col([results_1, results_2,\
                       results_3, results_4,\
                       results_5, results_6], float_format = '%.3f', \
                  model_names = ["ln_Occ_jt (1)", 
                                 "ln_Occ_jt (2)",
                                 "ln_ADR_jt (1)",
                                 "ln_ADR_jt (2)",
                                 "ln_RevPAR_jt (1)",
                                 "ln_RevPAR_jt (2)",
                                 
                                 ],\
                 
                    regressor_order = [   "2000-2004_round_up", "2005-2009_round_up","2010-2014_round_up","2015-2019_round_up",\
        "2000-2004_accum_rating", "2005-2009_accum_rating","2010-2014_accum_rating","2015-2019_accum_rating",\
        "2000-2004_accum_rating_sq", "2005-2009_accum_rating_sq","2010-2014_accum_rating_sq","2015-2019_accum_rating_sq",\
                                  ],\
                  info_dict={'N':lambda x: "{0:d}".format(int(x.nobs)),
                           'Adjusted R-squared':lambda x: "{:.2f}".format(x.rsquared_adj)},\
                  stars = True,drop_omitted=True)


beginningtex = """\\documentclass{report}
\\usepackage{booktabs}
\\begin{document}"""
endtex = "\end{document}"

output_dir = r"/Users/juju/Library/Mobile Documents/com~apple~CloudDocs/ta_project/output/ch1"
f = open(os.path.join(output_dir,'ch1_rdd_time_{}.tex'.format(i)), 'w')
f.write(beginningtex)
f.write(res_main.as_latex())
f.write(endtex)
f.close()

# By Chain

In [162]:
from statsmodels.api import add_constant
from linearmodels import IV2SLS
from statsmodels.iolib.summary2 import summary_col
import statsmodels.api as sm
import statsmodels
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing

OH = OneHotEncoder(handle_unknown='ignore')

fe_var= ["shareid","market_year_month","operation_year"]

x_fe = OH.fit_transform(df[fe_var]).toarray()
fe_var_cols = OH.fit(df[fe_var]).get_feature_names_out().tolist()

fe_var_df_1 = pd.DataFrame(x_fe, columns=fe_var_cols)


fe_var= ["Operation"]

x_fe = OH.fit_transform(df[fe_var]).toarray()
fe_var_cols = OH.fit(df[fe_var]).get_feature_names_out().tolist()

fe_var_df_2 = pd.DataFrame(x_fe, columns=fe_var_cols)
fe_var_df_2 = fe_var_df_2.rename(columns = {"Operation_Chain":"Chain",\
                                           "Operation_Independent":"Independent"})

con_var_list = ["accum_rating","round_up","accum_rating_sq"]
con_var_df = df[con_var_list]


x_temp = pd.concat([con_var_df,fe_var_df_2], axis=1).astype(float)
x_temp["Chain_rating"] = x_temp["Chain"]*x_temp["accum_rating"]
#x_temp["Chain_on_ta"] = x_temp["Chain"]*x_temp["on_ta"]
x_temp["Chain_round_up"] = x_temp["Chain"]*x_temp["round_up"]
x_temp["Chain_rating_sq"] = x_temp["Chain"]*x_temp["accum_rating_sq"]
#x_temp["Independent_rating"] = x_temp["Independent"]*x_temp["accum_rating"]
#x_temp["Chain_on_ta"] = x_temp["Chain"]*x_temp["on_ta"]
#x_temp["Independent_round_up"] = x_temp["Independent"]*x_temp["round_up"]
#x_temp["Independent_rating_sq"] = x_temp["Independent"]*x_temp["accum_rating_sq"]


con_var_df = x_temp[["Chain_rating","Chain_round_up",\
                     "accum_rating","round_up",\
]]

x_1= pd.concat([con_var_df,fe_var_df_1], axis=1).astype(float)
#x_1 = sm.add_constant(x_1)

In [163]:
con_var_df = x_temp[["Chain_rating","Chain_round_up","Chain_rating_sq",\
                  "accum_rating","round_up","accum_rating_sq"]]


x_2= pd.concat([con_var_df,fe_var_df_1], axis=1).astype(float)
#x_2 = sm.add_constant(x_2)

In [164]:
from statsmodels.iolib.summary2 import summary_col
from statsmodels.sandbox.regression.gmm import IV2SLS

model_1 = sm.OLS(y_1, x_1)
results_1 = model_1.fit()

model_2 = sm.OLS(y_1, x_2)
results_2 = model_2.fit()

model_3 = sm.OLS(y_2, x_1)
results_3 = model_3.fit()

model_4 = sm.OLS(y_2, x_2)
results_4 = model_4.fit()

model_5 = sm.OLS(y_3, x_1)
results_5 = model_5.fit()

model_6 = sm.OLS(y_3, x_2)
results_6 = model_6.fit()

In [165]:
res_main = summary_col([results_1, results_2, results_3, results_4,\
                       results_5, results_6], float_format = '%.3f', \
                  model_names = ["ln_Occ_jt (1)", 
                                 "ln_Occ_jt (2)",
                                 "ln_ADR_jt (1)",
                                 "ln_ADR_jt (2)",
                                 "ln_RevPAR_jt (1)",
                                 "ln_RevPAR_jt (2)",
                                 ],\
                 regressor_order = ["round_up",\
                                    "Chain_round_up",\
                                     "accum_rating",\
                                    "Chain_rating",\
                                    "accum_rating_sq",\
                                    "Chain_rating_sq",\
                                  ],\
                  info_dict={'N':lambda x: "{0:d}".format(int(x.nobs)),
                           'Adjusted R-squared':lambda x: "{:.2f}".format(x.rsquared_adj)},\
                  stars = True,drop_omitted=True)


beginningtex = """\\documentclass{report}
\\usepackage{booktabs}
\\begin{document}"""
endtex = "\end{document}"

output_dir = r"/Users/juju/Library/Mobile Documents/com~apple~CloudDocs/ta_project/output/ch1"
f = open(os.path.join(output_dir,'ch1_rdd_{}_chain.tex'.format(i)), 'w')
f.write(beginningtex)
f.write(res_main.as_latex())
f.write(endtex)
f.close()